In [2]:
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.log.exporter.xes import exporter as xes_exporter
from pm4py.statistics.traces.generic.log import case_statistics
from pm4py.objects.conversion.log import converter as log_converter

# ---- 1) Carregar o log ----
log = xes_importer.apply('log.xes')


# Converter log para DataFrame
df_log = log_converter.apply(log, variant=log_converter.Variants.TO_DATA_FRAME)

# Mostrar as primeiras linhas
df_log.head()



/home/kramer/Documents/UTFPR/Mineracao/codes/mineracao_de_processos_trabalho_final/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing log, completed traces :: 100%|██████████| 100000/100000 [00:22<00:00, 4537.61it/s]


,isCancelled,diagnosis,time:timestamp,caseType,speciality,org:resource,concept:name,blocked,isClosed,flagD,...,lifecycle:transition,case:concept:name,closeCode,actRed,actOrange,flagC,msgCount,version,msgType,msgCode
0,False,A,2012-12-16 19:33:10+00:00,A,A,ResA,NEW,False,True,True,...,complete,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2013-12-15 19:00:37+00:00,NaN,NaN,NaN,FIN,NaN,NaN,NaN,...,complete,A,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,2013-12-16 03:53:38+00:00,NaN,NaN,NaN,RELEASE,NaN,NaN,NaN,...,complete,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,2013-12-17 12:56:29+00:00,NaN,NaN,NaN,CODE OK,NaN,NaN,NaN,...,complete,A,NaN,False,False,False,0.0,A,NaN,NaN
4,NaN,NaN,2013-12-19 03:44:31+00:00,NaN,NaN,ResB,BILLED,NaN,NaN,NaN,...,complete,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
print(df_log.columns)

Index(['isCancelled', 'diagnosis', 'time:timestamp', 'caseType', 'speciality',
       'org:resource', 'concept:name', 'blocked', 'isClosed', 'flagD', 'flagB',
       'flagA', 'state', 'lifecycle:transition', 'case:concept:name',
       'closeCode', 'actRed', 'actOrange', 'flagC', 'msgCount', 'version',
       'msgType', 'msgCode'],
      dtype='object')


In [4]:
def segundos_para_legivel(segundos):
    anos, resto = divmod(segundos, 365*24*3600)       # 1 ano = 365 dias
    meses, resto = divmod(resto, 30*24*3600)          # 1 mês = 30 dias
    dias, resto = divmod(resto, 24*3600)
    horas, resto = divmod(resto, 3600)
    minutos, resto = divmod(resto, 60)
    segundos = resto
    return f"{int(anos)} anos, {int(meses)} meses, {int(dias)} dias, {int(horas)} horas, {int(minutos)} min"

In [5]:
import pandas as pd

# Calcular duração de cada caso diretamente do DataFrame
df_cases = df_log.groupby('case:concept:name')['time:timestamp'].agg(
    duracao_segundos=lambda x: (x.max() - x.min()).total_seconds()
).reset_index()

# Renomear colunas para ficar claro
df_cases.rename(columns={'case:concept:name': 'case_id'}, inplace=True)

# Conferir resultado
print(df_cases.head())
print(df_cases.columns)


# Calcular média, mínima e máxima
duracao_media = df_cases['duracao_segundos'].mean()
duracao_minima = df_cases['duracao_segundos'].min()
duracao_maxima = df_cases['duracao_segundos'].max()

# Exibir em formato legível
print("Duração média:", segundos_para_legivel(duracao_media))
print("Duração mínima:", segundos_para_legivel(duracao_minima))
print("Duração máxima:", segundos_para_legivel(duracao_maxima))



  case_id  duracao_segundos
0       A        31738281.0
1      AA        16713069.0
2     AAA               0.0
3    AAAA        11001024.0
4    AAAB         8955400.0
Index(['case_id', 'duracao_segundos'], dtype='object')
Duração média: 0 anos, 4 meses, 7 dias, 8 horas, 40 min
Duração mínima: 0 anos, 0 meses, 0 dias, 0 horas, 0 min
Duração máxima: 2 anos, 10 meses, 5 dias, 10 horas, 6 min


In [6]:
df_cases = df_cases[df_cases['duracao_segundos'] > 3600].reset_index(drop=True)

print(f"Número de casos após remover durações muito pequenas: {len(df_cases)}")

duracao_media = df_cases['duracao_segundos'].mean()
duracao_minima = df_cases['duracao_segundos'].min()
duracao_maxima = df_cases['duracao_segundos'].max()

print("Duração média:", segundos_para_legivel(duracao_media))
print("Duração mínima:", segundos_para_legivel(duracao_minima))
print("Duração máxima:", segundos_para_legivel(duracao_maxima))


Número de casos após remover durações muito pequenas: 76574
Duração média: 0 anos, 5 meses, 16 dias, 7 horas, 47 min
Duração mínima: 0 anos, 0 meses, 0 dias, 1 horas, 0 min
Duração máxima: 2 anos, 10 meses, 5 dias, 10 horas, 6 min


In [7]:
print(df_cases.columns)

Index(['case_id', 'duracao_segundos'], dtype='object')


In [8]:

df_cases.rename(columns={df_cases.columns[1]: 'duracao'}, inplace=True)
# Calcular limites usando percentis
limite_baixa = df_cases['duracao'].quantile(0.33)
limite_media = df_cases['duracao'].quantile(0.66)

# Função para classificar
def classificar_duracao(segundos):
    if segundos <= limite_baixa:
        return "Baixa"
    elif segundos <= limite_media:
        return "Média"
    else:
        return "Alta"

# Aplicar a função
df_cases['duracao'] = df_cases['duracao'].apply(classificar_duracao)

# Conferir resultado
df_cases.head()



,case_id,duracao
0,A,Alta
1,AA,Alta
2,AAAA,Média
3,AAAB,Média
4,AAAC,Baixa


In [9]:
# Lista dos atributos de caso
atributos_caso = ['caseType', 'diagnosis', 'speciality', 'state', 'blocked', 'isCancelled', 'isClosed']

# Para cada atributo, vamos pegar o valor do primeiro evento de cada caso
for attr in atributos_caso:
    df_attr = df_log[['case:concept:name', attr]].drop_duplicates(subset='case:concept:name')
    df_attr.rename(columns={'case:concept:name': 'case_id'}, inplace=True)
    
    # Fazer merge com df_cases
    df_cases = df_cases.merge(df_attr, on='case_id', how='left')

# Conferir o resultado
print(df_cases.columns)

Index(['case_id', 'duracao', 'caseType', 'diagnosis', 'speciality', 'state',
       'blocked', 'isCancelled', 'isClosed'],
      dtype='object')


In [10]:
# Selecionar apenas case_id e activity
df_atividades = df_log[['case:concept:name', 'concept:name']].copy()

# Renomear a coluna de case para merge
df_atividades.rename(columns={'case:concept:name': 'case_id', 'concept:name': 'atividade'}, inplace=True)

# Conferir primeiras linhas
df_atividades.head()


,case_id,atividade
0,A,NEW
1,A,FIN
2,A,RELEASE
3,A,CODE OK
4,A,BILLED


In [11]:
df_atividades_onehot = pd.get_dummies(df_atividades, columns=['atividade'], prefix='at_', prefix_sep='')

# Agrupar por case_id
df_atividades_onehot = df_atividades_onehot.groupby('case_id').sum().reset_index()

# Conferir resultado
df_atividades_onehot.head()


,case_id,at_BILLED,at_CHANGE DIAGN,at_CHANGE END,at_CODE ERROR,at_CODE NOK,at_CODE OK,at_DELETE,at_EMPTY,at_FIN,at_JOIN-PAT,at_MANUAL,at_NEW,at_REJECT,at_RELEASE,at_REOPEN,at_SET STATUS,at_STORNO,at_ZDBC_BEHAN
0,A,1,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0
1,AA,1,1,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0
2,AAA,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,AAAA,1,1,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0
4,AAAB,1,1,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0


In [12]:
# Merge com o DataFrame principal (que já tem atributos de caso e duracao)
df_with_activities = df_cases.merge(df_atividades_onehot, on='case_id', how='left')

# Conferir resultado
df_with_activities.head()

,case_id,duracao,caseType,diagnosis,speciality,state,blocked,isCancelled,isClosed,at_BILLED,...,at_FIN,at_JOIN-PAT,at_MANUAL,at_NEW,at_REJECT,at_RELEASE,at_REOPEN,at_SET STATUS,at_STORNO,at_ZDBC_BEHAN
0,A,Alta,A,A,A,In progress,False,False,True,1,...,1,0,0,1,0,1,0,0,0,0
1,AA,Alta,B,NaN,L,In progress,False,False,True,1,...,1,0,0,1,0,1,0,0,0,0
2,AAAA,Média,B,NaN,L,In progress,False,False,True,1,...,1,0,0,1,0,1,0,0,0,0
3,AAAB,Média,B,NaN,E,In progress,False,False,True,1,...,1,0,0,1,0,1,0,0,0,0
4,AAAC,Baixa,B,NaN,A,In progress,False,False,True,1,...,1,0,0,1,0,1,0,0,0,0


In [13]:
df_fluxo = df_log[['case:concept:name', 'time:timestamp', 'concept:name']].copy()
df_fluxo.rename(columns={'case:concept:name': 'case_id', 'concept:name': 'atividade'}, inplace=True)

df_fluxo = df_fluxo.sort_values(['case_id', 'time:timestamp'])


In [14]:
# Função para criar transições A->B
def criar_transicoes(atividades):
    return [f"{atividades[i]}->{atividades[i+1]}" for i in range(len(atividades)-1)]

# Agrupar por caso e criar lista de transições
df_transicoes = df_fluxo.groupby('case_id')['atividade'].apply(lambda x: criar_transicoes(list(x))).reset_index()


In [15]:
# Expandir lista de transições em linhas separadas
df_transicoes_exp = df_transicoes.explode('atividade')

# Criar one-hot das transições
df_transicoes_onehot = pd.get_dummies(df_transicoes_exp, columns=['atividade'], prefix='', prefix_sep='')

# Agrupar por case_id, somando os valores (cada caso terá 1 ou 0 em cada transição)
df_transicoes_onehot = df_transicoes_onehot.groupby('case_id').sum().reset_index()

# Conferir resultado
df_transicoes_onehot.head()


,case_id,BILLED->BILLED,BILLED->CODE NOK,BILLED->CODE OK,BILLED->JOIN-PAT,BILLED->MANUAL,BILLED->STORNO,CHANGE DIAGN->CHANGE DIAGN,CHANGE DIAGN->CHANGE END,CHANGE DIAGN->CODE OK,...,SET STATUS->REOPEN,SET STATUS->SET STATUS,STORNO->BILLED,STORNO->JOIN-PAT,STORNO->MANUAL,STORNO->REJECT,STORNO->REOPEN,STORNO->SET STATUS,STORNO->STORNO,ZDBC_BEHAN->BILLED
0,A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AAA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,AAAA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AAAB,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
df_final = df_with_activities.merge(df_transicoes_onehot, on='case_id', how='left')
df_final.head()

,case_id,duracao,caseType,diagnosis,speciality,state,blocked,isCancelled,isClosed,at_BILLED,...,SET STATUS->REOPEN,SET STATUS->SET STATUS,STORNO->BILLED,STORNO->JOIN-PAT,STORNO->MANUAL,STORNO->REJECT,STORNO->REOPEN,STORNO->SET STATUS,STORNO->STORNO,ZDBC_BEHAN->BILLED
0,A,Alta,A,A,A,In progress,False,False,True,1,...,0,0,0,0,0,0,0,0,0,0
1,AA,Alta,B,NaN,L,In progress,False,False,True,1,...,0,0,0,0,0,0,0,0,0,0
2,AAAA,Média,B,NaN,L,In progress,False,False,True,1,...,0,0,0,0,0,0,0,0,0,0
3,AAAB,Média,B,NaN,E,In progress,False,False,True,1,...,0,0,0,0,0,0,0,0,0,0
4,AAAC,Baixa,B,NaN,A,In progress,False,False,True,1,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# Agrupar por case_id e calcular início, fim e duração
df_temporal = df_log.groupby('case:concept:name')['time:timestamp'].agg(
    inicio='min',
    fim='max'
).reset_index()

df_temporal['duracao_segundos'] = (df_temporal['fim'] - df_temporal['inicio']).dt.total_seconds()

df_temporal.rename(columns={'case:concept:name': 'case_id'}, inplace=True)

df_temporal.head()


,case_id,inicio,fim,duracao_segundos
0,A,2012-12-16 19:33:10+00:00,2013-12-19 03:44:31+00:00,31738281.0
1,AA,2012-12-26 08:50:18+00:00,2013-07-07 19:21:27+00:00,16713069.0
2,AAA,2013-01-26 22:56:11+00:00,2013-01-26 22:56:11+00:00,0.0
3,AAAA,2013-06-18 20:26:32+00:00,2013-10-24 04:16:56+00:00,11001024.0
4,AAAB,2013-12-03 06:32:08+00:00,2014-03-16 22:08:48+00:00,8955400.0


In [18]:
df_final = df_final.merge(df_temporal, on='case_id', how='left')

df_final.head()


,case_id,duracao,caseType,diagnosis,speciality,state,blocked,isCancelled,isClosed,at_BILLED,...,STORNO->JOIN-PAT,STORNO->MANUAL,STORNO->REJECT,STORNO->REOPEN,STORNO->SET STATUS,STORNO->STORNO,ZDBC_BEHAN->BILLED,inicio,fim,duracao_segundos
0,A,Alta,A,A,A,In progress,False,False,True,1,...,0,0,0,0,0,0,0,2012-12-16 19:33:10+00:00,2013-12-19 03:44:31+00:00,31738281.0
1,AA,Alta,B,NaN,L,In progress,False,False,True,1,...,0,0,0,0,0,0,0,2012-12-26 08:50:18+00:00,2013-07-07 19:21:27+00:00,16713069.0
2,AAAA,Média,B,NaN,L,In progress,False,False,True,1,...,0,0,0,0,0,0,0,2013-06-18 20:26:32+00:00,2013-10-24 04:16:56+00:00,11001024.0
3,AAAB,Média,B,NaN,E,In progress,False,False,True,1,...,0,0,0,0,0,0,0,2013-12-03 06:32:08+00:00,2014-03-16 22:08:48+00:00,8955400.0
4,AAAC,Baixa,B,NaN,A,In progress,False,False,True,1,...,0,0,0,0,0,0,0,2014-06-30 15:25:03+00:00,2014-10-08 11:18:43+00:00,8625220.0


In [19]:
# Exportar para CSV
df_final.to_csv('transformed_log.csv', index=False)

print("Exportação para CSV concluída: transformed_log.csv")

Exportação para CSV concluída: transformed_log.csv
